In [1]:
import networkx as nx
from collections import defaultdict
from itertools import combinations
from bs4 import BeautifulSoup  
import json
import pandas as pd
import concurrent.futures 
import requests
import math
import csv
from collections import Counter
import ast
import netwulf as nw
import matplotlib.pyplot as plt
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [48]:
dictionaries = []
def process_data(date):
    link = "https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/"
    link += date
    chrome_prefs = {
        "profile.default_content_settings.cookies": 1  
    }
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", chrome_prefs)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(link)
    driver.implicitly_wait(15)
    table = driver.find_element(By.XPATH, '//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table')
    rows = table.find_elements(By.TAG_NAME, 'tr')
    rows.pop(0)
    weather_data = {}
    attributes = ['Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.', 'Condition']
    
    for row in rows:
        if(len(row.text)>3):
            parts = row.text.strip().split(" ")
            
            time = ' '.join(parts[:2])
            details = {
                attributes[0]: parts[2] + ' ' + parts[3],  # Temperature
                attributes[1]: parts[4] + ' ' + parts[5],  # Dew Point
                attributes[2]: parts[6] + ' ' + parts[7],  # Humidity
                attributes[3]: parts[8],                   # Wind
                attributes[4]: parts[9],                   # Wind Speed
                attributes[5]: parts[11],                  # Wind Gust
                attributes[6]: parts[13] + ' ' + parts[14],# Pressure
                attributes[7]: parts[15] + ' ' + parts[16],# Precip.
                attributes[8]: ' '.join(parts[17:])                   # Condition
                }
            weather_data[time] = details
    driver.quit()
    dictionaries.append(weather_data)


In [94]:
dates = []
#dictionaries = []
for i in range(31):
    day = i+1
    print(day)
    dates.append("2023-5-"+str(day))  

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [12]:
with open('data.json', 'r') as file:
    data = json.load(file)

year_count = 2013
annual_data = {}
current_year_data = {}
incident_count = 1

for entry in data:
    current_year_data[str(incident_count)] = entry
    incident_count += 1

    if incident_count > 31:
        annual_data[str(year_count)] = current_year_data
        year_count += 1
        current_year_data = {}
        incident_count = 1

if current_year_data:
    annual_data[str(year_count)] = current_year_data

""" with open('transformed_data.json', 'w') as file:
    json.dump(annual_data, file, indent=4) """

" with open('transformed_data.json', 'w') as file:\n    json.dump(annual_data, file, indent=4) "

In [ ]:
with open('transformed_data.json', 'r') as file:
    data = json.load(file)
average_per_year = []
for year in data.items():
    average_per_month = []
    for day in year[1].items():
        sum = 0.0
        for time in day[1].items():
            precip = time[1]["Precip."].strip("in").strip(" ")
            sum += float(precip)
        average = float(sum/len(day))
        average_per_month.append(average)
    average_per_year.append(average_per_month)



In [81]:
for year in average_per_year:
    print(year)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.150000000000001, 0.5, 0.0, 0.8999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.5, 0.05, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0]
[0.39999999999999997, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.05, 1.35, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15000000000000002, 0.6000000000000001, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9500000000000004]
[0.0, 0.0, 0.65, 0.0, 0.0, 0.65, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.15000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2000000000000002, 0.0]
[0.0, 0.0, 0.0, 0.0, 2.6500000000000004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1500000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.44999999999999996, 0.3, 0.0, 0.0, 0.05, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [78]:
max_difference = []
for j in range(len(average_per_year[0])):
    temp_max = 0.0
    max_index = []
    temp_min = 1.0
    min_index = []
    for i in range(len(average_per_year)):
        if (float(average_per_year[i][j]) > temp_max):
            temp_max = average_per_year[i][j]
            max_index = [i,j]
        if float(average_per_year[i][j]) < temp_min:
            temp_min = average_per_year[i][j]
            min_index = [i,j]
    difference = temp_max-temp_min
    max_difference.append([difference,max_index,min_index])

    


In [79]:
print(max_difference)

[[0.39999999999999997, [1, 0], [0, 0]], [0.0, [7, 1], [0, 1]], [0.0, [7, 2], [0, 2]], [0.0, [7, 3], [0, 3]], [0.0, [7, 4], [0, 4]], [0.0, [7, 5], [0, 5]], [0.0, [7, 6], [0, 6]], [0.25, [1, 7], [2, 7]], [0.0, [7, 8], [1, 8]], [1.0, [1, 9], [0, 9]], [0.05, [7, 10], [1, 10]], [0.0, [7, 11], [0, 11]], [0.0, [7, 12], [0, 12]], [0.0, [7, 13], [0, 13]], [0.0, [7, 14], [0, 14]], [1.35, [1, 15], [0, 15]], [0.0, [7, 16], [0, 16]], [0.0, [7, 17], [0, 17]], [0.0, [7, 18], [2, 18]], [0.0, [7, 19], [0, 19]], [0.0, [7, 20], [0, 20]], [0.15000000000000002, [1, 21], [0, 21]], [0.6000000000000001, [1, 22], [2, 22]], [0.2, [1, 23], [2, 23]], [0.0, [7, 24], [0, 24]], [0.0, [7, 25], [0, 25]], [0.0, [7, 26], [0, 26]], [0.0, [7, 27], [2, 27]], [0.0, [7, 28], [0, 28]], [0.0, [7, 29], [0, 29]], [0.0, [7, 30], [0, 30]]]
